# Testing core behaviour of the IVP solver module


Some quick 'examples' that should give sensible results and not error if everything is working properly.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sympy as sm
sm.init_printing()

## Scaling from one to many dimensions

In [2]:
from pylab import *
from spylind import spyIVP as svp
from spylind import spylind as spl # master equation useful bits
import qutip as q

Tensorflow not accessible: No module named 'backend_tf'
No pretty printing stuff (probably because no Pandas)


In [3]:
# OD
xS, yS, vxS, vyS, kS = sm.symbols("x, y, v_x, v_y, k", real=True)
g= -10.0
dy_dtD = {xS:vxS, 
       yS:vyS,
       vxS:0,
       vyS:g -10*kS}
sys = svp.ODESys(dy_dtD, dimsD={})
sys.set_initial_conditions({xS:0,yS:np.random.normal(sys.dims.shape),vxS:10,vyS:10}) 
model = sys.setup_model(subsD = {kS:0})
tSteps = linspace(0,2,30)
arr = model.integrate(tSteps)

#sm.ode
plot(tSteps, arr.real)
xlabel('t')
legend(dy_dtD.keys())



TypeError: ODESys.__init__() got an unexpected keyword argument 'dimsD'

In [177]:
# Add an 'mass' and 'index' dimension
MS, indS = sm.symbols("M, I", real=True) 
sys = svp.ODESys(dy_dtD,  dims={MS:linspace(.3,1,10), indS: arange(5)})
display(sys.state_shape, sys.dim_shape)

[autoreload of spylind.spyIVP failed: Traceback (most recent call last):
  File "/home/morgan/miniforge3/envs/spylind/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/morgan/miniforge3/envs/spylind/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/morgan/miniforge3/envs/spylind/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/morgan/miniforge3/envs/spylind/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, i

TypeError: __init__() got an unexpected keyword argument 'dims'

In [176]:
%debug

> /home/morgan/dev_local/spylind/spylind/spyIVP.py(105)__init__()
    103         self.dy_dtD = dy_dtD
    104         self.sim_size = int(len(dy_dtD) * np.product(self.dims.shape))
--> 105         self.state_shape = tuple(len(dy_dtD), *dims.shape)
    106         self.state_func_signature = state_func_signature
    107 



ipdb>  dims


Munch({'values': [], 'symbols': [], 'shape': 1, 'broadcastable': {}})


ipdb>  q


In [148]:
sys.dims

[array([[0.3       ],
        [0.37777778],
        [0.45555556],
        [0.53333333],
        [0.61111111],
        [0.68888889],
        [0.76666667],
        [0.84444444],
        [0.92222222],
        [1.        ]]),
 array([[0, 1, 2, 3, 4]])]

In [154]:
import pdb

In [156]:
#Initialise with a mix of single numbers and full-sized arrays
sys.set_initial_conditions({xS:0, yS:linspace(-1,1,10)[:,newaxis]*np.ones(sys.dim_shape), vxS:10, vyS:15 + 0*sys.dims[0] +0*sys.dims[1] }) 
display(sys.par0.shape)
def couplingF(t, dimAxes, state, driving_vals, MAxInd = sys.dimSyms.index(MS)):
    M_ax = dimAxes[MAxInd]
    x,y,vx,vy = state #prop_state_syms 
    COM = np.sum(y*M_ax)/M_ax.sum()
    pdb.set_trace()
    #print(COM)
    return (y-COM)/M_ax
sys.set_state_dependence({kS:couplingF})

model= sys.setup_model()#bForceStateDimensions=True)
tSteps = linspace(0,3.,5000)
arr = model.integrate(tSteps)


plot(tSteps, arr[:,1,:].mean(axis=-1).real)
xlabel('t')
ylabel('y')
title("attraction toward the COM")


Not integrating first step (it's just the initial state)
> <ipython-input-156-5a4747f0275d>(10)couplingF()
      8     pdb.set_trace()
      9     #print(COM)
---> 10     return (y-COM)/M_ax
     11 sys.set_state_dependence({kS:couplingF})
     12 



ipdb>  (y-COM).shape


(10, 5)


ipdb>  y-COM


array([[-2.0968661 +0.j, -2.0968661 +0.j, -2.0968661 +0.j,
        -2.0968661 +0.j, -2.0968661 +0.j],
       [-1.87464387+0.j, -1.87464387+0.j, -1.87464387+0.j,
        -1.87464387+0.j, -1.87464387+0.j],
       [-1.65242165+0.j, -1.65242165+0.j, -1.65242165+0.j,
        -1.65242165+0.j, -1.65242165+0.j],
       [-1.43019943+0.j, -1.43019943+0.j, -1.43019943+0.j,
        -1.43019943+0.j, -1.43019943+0.j],
       [-1.20797721+0.j, -1.20797721+0.j, -1.20797721+0.j,
        -1.20797721+0.j, -1.20797721+0.j],
       [-0.98575499+0.j, -0.98575499+0.j, -0.98575499+0.j,
        -0.98575499+0.j, -0.98575499+0.j],
       [-0.76353276+0.j, -0.76353276+0.j, -0.76353276+0.j,
        -0.76353276+0.j, -0.76353276+0.j],
       [-0.54131054+0.j, -0.54131054+0.j, -0.54131054+0.j,
        -0.54131054+0.j, -0.54131054+0.j],
       [-0.31908832+0.j, -0.31908832+0.j, -0.31908832+0.j,
        -0.31908832+0.j, -0.31908832+0.j],
       [-0.0968661 +0.j, -0.0968661 +0.j, -0.0968661 +0.j,
        -0.0968661 +0.j

...

array([[-1.        +0.j, -1.        +0.j, -1.        +0.j,
        -1.        +0.j, -1.        +0.j],
       [-0.77777778+0.j, -0.77777778+0.j, -0.77777778+0.j,
        -0.77777778+0.j, -0.77777778+0.j],
       [-0.55555556+0.j, -0.55555556+0.j, -0.55555556+0.j,
        -0.55555556+0.j, -0.55555556+0.j],
       [-0.33333333+0.j, -0.33333333+0.j, -0.33333333+0.j,
        -0.33333333+0.j, -0.33333333+0.j],
       [-0.11111111+0.j, -0.11111111+0.j, -0.11111111+0.j,
        -0.11111111+0.j, -0.11111111+0.j],
       [ 0.11111111+0.j,  0.11111111+0.j,  0.11111111+0.j,
         0.11111111+0.j,  0.11111111+0.j],
       [ 0.33333333+0.j,  0.33333333+0.j,  0.33333333+0.j,
         0.33333333+0.j,  0.33333333+0.j],
       [ 0.55555556+0.j,  0.55555556+0.j,  0.55555556+0.j,
         0.55555556+0.j,  0.55555556+0.j],
       [ 0.77777778+0.j,  0.77777778+0.j,  0.77777778+0.j,
         0.77777778+0.j,  0.77777778+0.j],
       [ 1.        +0.j,  1.        +0.j,  1.        +0.j,
         1.        +0.j

ipdb>  COM


(1.0968660968660964+0j)


ipdb>  q


BdbQuit: 

## Changing between real-only and complex models

## Robust changing between backends